In [1]:
# Basic imports
import os,sys,re
from importlib import reload
import numpy as np
import pandas as pd
#import torch
print(os.getpid())

3168653


In [2]:
# ChromAn related imports
sys.path.append(r'/lab/weissman_imaging/puzheng/Softwares/') # parent folder of ChromAn
import ImageAnalysis3 as ia3
import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast
from ChromAn.src import file_io

from ChromAn.src.file_io import dax_process
from ChromAn.src.file_io import data_organization
from ChromAn.src.visual_tools import interactive
from ChromAn.src.correction_tools.alignment import generate_drift_crops
from ChromAn.src.spot_tools.match_spots import colocalize_spots

import pickle

In [3]:
# data folder
data_folder = r'/lab/weissman_imaging/puzheng/PE_LT/20230913-4T1top8_preEdit_400k0907'
# scan subfolders
folders, fovs = data_organization.search_fovs_in_folders(data_folder)
# analysis folder
analysis_folder = os.path.join(data_folder, 'Analysis')
# figure_folder
figure_folder = os.path.join(analysis_folder, 'saved_figures_20231018')
if not os.path.exists(figure_folder):
    os.makedirs(figure_folder)

- searching in folder: /lab/weissman_imaging/puzheng/PE_LT/20230913-4T1top8_preEdit_400k0907
-- 17 folders, 325 fovs detected.


In [4]:
color_usage_df = data_organization.Color_Usage(os.path.join(analysis_folder, "Color_Usage.csv"))

- load color_usage from file: /lab/weissman_imaging/puzheng/PE_LT/20230913-4T1top8_preEdit_400k0907/Analysis/Color_Usage.csv


In [5]:
save_folder = analysis_folder
save_filenames = [os.path.join(save_folder, _fl) for _fl in os.listdir(save_folder)
                  if _fl.split(os.extsep)[-1]=='hdf5']
import re
match = re.match(r'.*_([0-9]+).hdf5', os.path.basename(save_filenames[0]), ).groups()[0]
save_filenames = [_f for _f in sorted(save_filenames, 
                                      key=lambda _v:int(re.match(r'.*_([0-9]+).hdf5', os.path.basename(_v), ).groups()[0]))]

In [6]:
processed_filenames = [os.path.join(save_folder, _fl) for _fl in os.listdir(save_folder)
                       if '_processed_info.pkl' in _fl]
processed_filenames = [_f for _f in sorted(processed_filenames, 
                                      key=lambda _v:int(re.match(r'.*_([0-9]+)_processed_info.pkl', os.path.basename(_v), ).groups()[0]))]


In [7]:
# load infos:
all_intbc_ims = []
all_edit_ims = []
for _fl in processed_filenames:
    #print(_fl)
    _fov_info = pickle.load(open(_fl, 'rb'))
    for _info in _fov_info:
        all_intbc_ims.append(_info['intbc_ims'])
        all_edit_ims.append(_info['edit_ims'])
    # extract all edits ims
    print(f" loaded {len(_fov_info)} from file: {os.path.basename(_fl)}")
print(f"- {len(all_intbc_ims)} in total")

 loaded 47 from file: Conv_zscan_003_processed_info.pkl
 loaded 63 from file: Conv_zscan_004_processed_info.pkl
 loaded 23 from file: Conv_zscan_005_processed_info.pkl
 loaded 62 from file: Conv_zscan_006_processed_info.pkl
 loaded 58 from file: Conv_zscan_007_processed_info.pkl
 loaded 53 from file: Conv_zscan_008_processed_info.pkl
 loaded 11 from file: Conv_zscan_009_processed_info.pkl
 loaded 58 from file: Conv_zscan_010_processed_info.pkl
 loaded 34 from file: Conv_zscan_011_processed_info.pkl
 loaded 30 from file: Conv_zscan_012_processed_info.pkl
 loaded 89 from file: Conv_zscan_013_processed_info.pkl
 loaded 38 from file: Conv_zscan_014_processed_info.pkl
 loaded 25 from file: Conv_zscan_015_processed_info.pkl
 loaded 29 from file: Conv_zscan_016_processed_info.pkl
 loaded 51 from file: Conv_zscan_017_processed_info.pkl
 loaded 54 from file: Conv_zscan_020_processed_info.pkl
 loaded 71 from file: Conv_zscan_021_processed_info.pkl
 loaded 61 from file: Conv_zscan_022_processed_i

In [38]:
len(all_intbc_ims)

47

In [43]:
# Save to temp location
temp_folder = r'/lab/solexa_weissman/puzheng/PE_LT/Analysis/0913_top8'
if not os.path.isdir(temp_folder):
    os.makedirs(temp_folder)
# ims files:
intbcIm_filename = os.path.join(temp_folder, 'intbc_ims.npy')
editIm_filename = os.path.join(temp_folder, 'edit_ims.npy')
print(intbcIm_filename, editIm_filename)
np.save(intbcIm_filename, all_intbc_ims)
np.save(editIm_filename, all_edit_ims)


/lab/solexa_weissman/puzheng/PE_LT/Analysis/0913_top8/intbc_ims.npy /lab/solexa_weissman/puzheng/PE_LT/Analysis/0913_top8/edit_ims.npy


In [9]:
# reload from here:
temp_folder = r'/lab/solexa_weissman/puzheng/PE_LT/Analysis/0913_top8'
intbcIm_filename = os.path.join(temp_folder, 'intbc_ims.npy')
editIm_filename = os.path.join(temp_folder, 'edit_ims.npy')
print(intbcIm_filename, editIm_filename)
all_intbc_ims = np.load(intbcIm_filename, )
all_edit_ims = np.load(editIm_filename, )


/lab/solexa_weissman/puzheng/PE_LT/Analysis/0913_top8/intbc_ims.npy /lab/solexa_weissman/puzheng/PE_LT/Analysis/0913_top8/edit_ims.npy


# Load codebook

In [45]:
codebook_filename = r'/lab/solexa_weissman/puzheng/MERFISH_Probes/PE_TS/NewTargetSites/IntBC_MERFISH/full_codebook.csv'
codebook = pd.read_csv(codebook_filename)

In [ ]:
# mimic MERLin Optimization, prepare data:
class Optimization_decode():
    """
    An analysis class for decoding MERFISH data
    """
    def __init__(self, codebook_filename, 
                 color_usage_filename=None,
                 verbose=True,
                 ):
        self.codebook_filename = codebook_filename
        self._load_codebook()
        self.color_usage_filename = color_usage_filename
        self._load_color_usage()

    def _load_color_usage(self):
        """
        Load color usage from file
        """
        if self.color_usage_df is None:
            if self.verbose:
                print(f" - loading color usage from file: {self.color_usage_filename}")
            self.color_usage_df = data_organization.Color_Usage(self.color_usage_filename)
        else:
            pass
    
    def _load_codebook(self):
        """
        Load codebook from file
        """
        if self.codebook is None:
            if self.verbose:
                print(f" - loading codebook from file: {self.codebook_filename}")
            self.codebook = pd.read_csv(self.codebook_filename)
        else:
            pass
    
    def _load_images(self, cropped_ims):
        """
        include loaded images into the class, for now its a test case
        """
        # add loaded images
        self.ims = cropped_ims
        return
    
    def _init_scale_factors(self, ) -> np.ndarray:
        """
        initialize scale factors
        """
        self.scale_factors = np.ones((self.ims.shape[0], self.ims.shape[1]), dtype=np.float32)
        
        
        